Model simulation Strain_3 (Knock-out gltB, gltD).By knockint-out gltB and gltD the consumption of 2-oxoglutarate is inhibited by the absence of the correlated reaction.

In [40]:
from cobra.io import read_sbml_model
model = read_sbml_model('modified_model.xml')
model

Name,iJO1366
Memory address,1f955fe4250
Number of metabolites,1810
Number of reactions,2592
Number of genes,1368
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iJO1366_core_53p95M - 1.0*BIOMASS_Ec_iJO1366_core_53p95M_reverse_5c8b1
Compartments,"cytosol, extracellular space, periplasm"


Based on the paper another knock-out that will increase 2-oxoglutarate concentration is gltBD 


In [41]:
# research of gltB gene in our model
gltB = None
for gene in model.genes:
    if "gltB" in gene.name:
        gltB = gene.id
        break

print("Gene ID for gltB in model:", gltB)

Gene ID for gltB in model: b3212


In [42]:
model.genes.b3212

Gene identifier,b3212
Name,gltB
Memory address,0x1f956b8f110
Functional,True
In 1 reaction(s),GLUSy


In [43]:
# research of gltD gene in our model
gltD = None
for gene in model.genes:
    if "gltD" in gene.name:
        gltD = gene.id
        break

print("Gene ID for gltB in model:", gltD)

Gene ID for gltB in model: b3213


In [44]:
model.genes.b3213

Gene identifier,b3213
Name,gltD
Memory address,0x1f956b8e510
Functional,True
In 1 reaction(s),GLUSy


Let's search for the connected reaction to konck it out

In [45]:
model.reactions.GLUSy

Reaction identifier,GLUSy
Name,Glutamate synthase (NADPH)
Memory address,0x1f95747b5d0
Stoichiometry,akg_c + gln__L_c + h_c + nadph_c --> 2.0 glu__L_c + nadp_c 2-Oxoglutarate + L-Glutamine + H+ + Nicotinamide adenine dinucleotide phosphate - reduced --> 2.0 L-Glutamate + Nicotinamide adenine dinucleotide phosphate
GPR,b3212 and b3213
Lower bound,0.0
Upper bound,1000.0


Test the production of EFE reaction before the knock-out

In [46]:
with model:
    solution = model.optimize()
    print('Maximum theoretical biomass productivity:', solution.fluxes['BIOMASS_Ec_iJO1366_core_53p95M'], '/h')

    model.objective = model.reactions.EFE_m
    E_production = model.optimize().objective_value
    print('Maximum theoretical productivity of Ethylene_forming_enzyme_main reaction:', E_production, 'mmol/gDW*h')

Maximum theoretical biomass productivity: 0.9823718127269851 /h
Maximum theoretical productivity of Ethylene_forming_enzyme_main reaction: 12.780573951434883 mmol/gDW*h


Knock-out of the reaction connected to gltBD genes

In [47]:
model.reactions.GLUSy.knock_out()
model.reactions.GLUSy

Reaction identifier,GLUSy
Name,Glutamate synthase (NADPH)
Memory address,0x1f95747b5d0
Stoichiometry,akg_c + gln__L_c + h_c + nadph_c --> 2.0 glu__L_c + nadp_c 2-Oxoglutarate + L-Glutamine + H+ + Nicotinamide adenine dinucleotide phosphate - reduced --> 2.0 L-Glutamate + Nicotinamide adenine dinucleotide phosphate
GPR,b3212 and b3213
Lower bound,0
Upper bound,0


Maximum theoretical biomass and EFE productivity after the knock-out

In [48]:
with model:
    solution = model.optimize()
    print('Maximum theoretical biomass productivity:', solution.fluxes['BIOMASS_Ec_iJO1366_core_53p95M'], '/h')

    model.objective = model.reactions.EFE_m
    E_production = model.optimize().objective_value
    print('Maximum theoretical productivity of Ethylene_forming_enzyme_main reaction:', E_production, 'mmol/gDW*h')

Maximum theoretical biomass productivity: 0.9823718127269903 /h
Maximum theoretical productivity of Ethylene_forming_enzyme_main reaction: 12.78057395143489 mmol/gDW*h


Combination of the new strain with media implementation

In [49]:
YEPDmedium = model.medium
YEPDmedium['EX_glc__D_e'] = 0
YEPDmedium['EX_sucr_e'] = 40
YEPDmedium['EX_ala__L_e'] = 20
YEPDmedium['EX_arg__L_e'] = 20
YEPDmedium['EX_asn__L_e'] = 20
YEPDmedium['EX_cys__L_e'] = 20
YEPDmedium['EX_gln__L_e'] = 20
YEPDmedium['EX_glu__L_e'] = 20
YEPDmedium['EX_gly_e'] = 0
YEPDmedium['EX_his__L_e'] = 0
YEPDmedium['EX_ile__L_e'] = 0
YEPDmedium['EX_leu__L_e'] = 0
YEPDmedium['EX_met__L_e'] = 0
YEPDmedium['EX_phe__L_e'] = 0
YEPDmedium['EX_pro__L_e'] = 20
YEPDmedium['EX_ser__L_e'] = 20
YEPDmedium['EX_thr__L_e'] = 20
YEPDmedium['EX_trp__L_e'] = 20
YEPDmedium['EX_tyr__L_e'] = 0
YEPDmedium['EX_val__L_e'] = 0
YEPDmedium['EX_lys__L_e'] = 0
YEPDmedium['EX_asp__L_e'] = 20

In [50]:
with model:
    model.medium = YEPDmedium
    solution = model.optimize()
    print('Statistically best (With Sucrose instead of Glucose):')
    print('Maximum theoretical growth-rate:', solution.fluxes['BIOMASS_Ec_iJO1366_core_53p95M'], '/h')
    

    model.objective = model.reactions.EFE_m
    E_production = model.optimize().objective_value
    print('Maximum theoretical productivity of Ethylene_forming_enzyme_main reaction:', E_production, 'mmol/gDW*h')

Statistically best (With Sucrose instead of Glucose):
Maximum theoretical growth-rate: 19.191474500959977 /h
Maximum theoretical productivity of Ethylene_forming_enzyme_main reaction: 277.0509433962263 mmol/gDW*h


Maximum theoretical EFE productivity and yield after the knock-out without the media implementation

In [51]:
model.objective = model.reactions.eth_tp
eth_production = model.optimize().objective_value
print('Maximum productivity =', eth_production, 'mmol/gDW*h')
   
eth_maximum_yield = eth_production / (-1*(model.reactions.EX_glc__D_e.flux))
print('Maximum theoretical yield =', eth_maximum_yield, 'mmol-eth/mmol-glc')

Maximum productivity = 12.780573951434846 mmol/gDW*h
Maximum theoretical yield = 1.2780573951434846 mmol-eth/mmol-glc
